In [1]:
#include <stdlib.h>
#include <iostream>
#include <set>
#include "infoapi.hpp"

struct Ticket: public Record{
    string name;
    int    number;
    void on_save(ostream&out){ out << number << endl << name << endl; }
    void on_load(istream&in) { in >> number >> name; }
};

enum OP { GET_TICKET, PRINT_TICKETS };

class TicketsDatabase: public DatabaseWorker{
public:
    TicketsDatabase(unsigned pid, EventLog&log):DatabaseWorker(pid,log){
        for(int n=1; n<=20; n++) vacant_tickets.insert(n);
    }
private:
    void on_run(int tag)override{
        switch(tag)
        {
            case OP::GET_TICKET:
            {
                Ticket my_ticket;
IO_SECTION_BEGIN
                cout << "Please, print your name:" << endl; 
                cin >> my_ticket.name;
IO_SECTION_END
IO_SYNC_POINT(my_ticket)
                if(selected_tickets.find(my_ticket.name)!=selected_tickets.end()){
IO_SECTION_BEGIN
                    cout << "Your already took the ticket number:" << selected_tickets[my_ticket.name] << endl;
IO_SECTION_END    
                    return;
                }
                for(;;){ // trying to get a ticket
                    int size = vacant_tickets.size();
                    if(size==0){
IO_SECTION_BEGIN
                        cout << "Your are late. No more tickets to select." << endl;
IO_SECTION_END
                        return;
                    }              
                    int selected = rand()%size;
                    set<int>::iterator it = vacant_tickets.begin();
                    for(int i=0; i!=selected; i++,it++);                    
                    my_ticket.number = *it;
                    
IO_SYNC_POINT(my_ticket)

                    if(vacant_tickets.find(my_ticket.number)==vacant_tickets.end()) 
                        continue; // one more try, someone intercepted my ticket
                    // good, no one intercepted my ticket
                    vacant_tickets.erase(my_ticket.number);    
                    selected_tickets[my_ticket.name] = my_ticket.number;
IO_SECTION_BEGIN                
                    cout << "Your ticket number is:" << my_ticket.number;
IO_SECTION_END                     
                    return;           
                }
            }
            case OP::PRINT_TICKETS:
            {
IO_SECTION_BEGIN
                for(auto& t: selected_tickets){
                    cout << "         Name:" << t.first  << endl
                         << "Ticket number:" << t.second << endl << endl;
                }
IO_SECTION_END
            }
        }
    }
    set<int>        vacant_tickets;
    map<string,int> selected_tickets;
};

EventLog event_log;

In [2]:
{
    TicketsDatabase tickets(0,event_log);
    tickets.run(OP::GET_TICKET);
}

In [3]:
{
    TicketsDatabase tickets(1,event_log);
    tickets.run(OP::PRINT_TICKETS);
}